<a href="https://colab.research.google.com/github/rajagopalmotivate1/temp1/blob/master/NEST_PubMed_v37.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NESTLE ASL Platform  **
## HCP Solution Platform Leader: Karenne  

# Goals: 
## 1. To enable BU enduser to query pubmed based on MeSH / QA, 
## 2. For the query, Idenitfy list of frequently used keywords or automatically extracted key phrases using Bio BERT or Clinical BERT
## 3. Find HCPs: Find HCPs in twitter based on these keywords/key phrases 
## 4. Find Content: Find tweets related to these keywords 
## 5. Find Content: Find content from pubMed


 



#Install components that need restart of run time

In [1]:
!pip install -U spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bc5cdr_md-0.2.4.tar.gz

     |████████████████████████████████| 10.0MB 3.2MB/s 
     |████████████████████████████████| 2.1MB 47.8MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.0MB 320kB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
     |████████████████████████████████| 296kB 32.0MB/s 
     |████████████████████████████████| 17.0MB 1.4MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.2.4-cp36-none-any.whl size=17161110 sha256=a93b4bbbd436c609cb3c1e93e0690db8fdd07b7a83b6297036d0ec399edf19b7
  Stored in directory: /root/.cache/pip/wheels/34/60/b9/fabd9c3eeba17ed66df745479f2fc502a6702755cb4a9632f2
Successfully built en-core-sci-sm
     |████████████████████████████████| 70.1MB 55kB/s 
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.2.

# Define class & methods  

In [1]:

import json
import pandas as pd
import os
import time


In [2]:

import re
import numpy as np

In [3]:
from google.colab import files 

##Define spacy related methods

## Define BERT and spacy related methods

In [4]:
!pip install -q transformers

     |████████████████████████████████| 778kB 3.2MB/s 
     |████████████████████████████████| 1.1MB 7.9MB/s 
     |████████████████████████████████| 3.0MB 10.3MB/s 
     |████████████████████████████████| 890kB 18.2MB/s 


In [5]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

In [6]:
def getNERBioBERT_Init(optionnumber):
    #myinputstringclinical = "In the field of Cardiovascular risk in London ,  heart murmur are harmless but may be abnormal too" 

  #bioNER = pipeline('ner' )
  #bioNER = pipeline('ner', model="emilyalsentzer/Bio_ClinicalBERT", tokenizer="emilyalsentzer/Bio_ClinicalBERT")
  if optionnumber == 0: 
    bioNER = pipeline('ner' )

  if optionnumber == 1: 
    bioNER = pipeline('ner', model="monologg/biobert_v1.1_pubmed", tokenizer="monologg/biobert_v1.1_pubmed")
  #bioNER = pipeline('ner',  model="clagator/biobert_squad2_cased",  tokenizer="clagator/biobert_squad2_cased" )
  #GOOD
  if optionnumber == 2: 
    bioNER = pipeline('ner',  model="simonlevine/Bio_ClinicalBERT-4096",  tokenizer="simonlevine/Bio_ClinicalBERT-4096" )

  if optionnumber == 3: 
    bioNER = pipeline('ner', model="emilyalsentzer/Bio_ClinicalBERT", tokenizer="emilyalsentzer/Bio_ClinicalBERT")

  if optionnumber == 4: 
    bioNER = pipeline('ner', model="fran-martinez/scibert_scivocab_cased_ner_jnlpba", tokenizer="fran-martinez/scibert_scivocab_cased_ner_jnlpba")


  return bioNER

In [7]:
def getNERBioBERT(myinputstringclinical) : 

  myinputstringclinical1 = myinputstringclinical.replace("-", " ")

  datadictionary = bioNER( myinputstringclinical1 )

  ans = getLogicalPhrasesFromWordpieces(datadictionary, myinputstringclinical1 )
  return ans
  

In [8]:
def getLogicalPhrasesFromWordpieces( myArrayofDictionariesNER, myinputstring ):
  myentities = [] 
  myentitiesNew = [] 

  for aDictionary in myArrayofDictionariesNER:
   # print(aDictionary)
    word = aDictionary['word']
    if (word[:2] == '##') : 
      temp = myentities[len(myentities)-1]
      myentities[len(myentities)-1] = temp +  word[2:]    
    else:
      myentities.append(word)

  if len(myentities) > 0:
    myentitiesNew.append(myentities[0])
    for i in range(1, len(myentities) ):
      concactWord = myentitiesNew[len(myentitiesNew)-1] + ' ' + myentities[i]
      myinputstring1 = myinputstring.replace("-", " ")
    #  print(concactWord)
      if (concactWord.lower() in myinputstring1.lower()) :
        myentitiesNew[len(myentitiesNew)-1] = concactWord
      else:
        myentitiesNew.append(myentities[i])  

  myentitiesNewCleaned = []
  for a in myentitiesNew:
    if not ((a == '[CLS]') or (a == '[SEP]') or (a == ',') or (a == '.')):
      myentitiesNewCleaned.append(a)

  return myentitiesNewCleaned

##Define PubMED related methods 

In [9]:
!pip install pymed

In [10]:
from pymed import PubMed

### Preview scrape in pubmed




In [11]:
pubmed = PubMed(tool="NESTLE_ASL__HCP_Solution", email="name@email.com")


In [12]:
def scrape_save_count(query):
  numberofresults = pubmed.getTotalResultsCount(query )
  return numberofresults

In [13]:
def previewScrape():
  print('For your search query, Here is the number of pubmed search articles that can be scraped')
  print( 'no of articles' + '\t \t \t \t \t \t : \t\t\t' +  'MeSH query' )
  for agegroup in MeSHlistbyAgeGroup:
    query = "(" +  agegroup + "[MeSH Major Topic])"
    count = scrape_save_count(query)
    time.sleep(2)
    print( str(count) + '\t \t \t \t \t \t : \t\t\t' + str(query) )


### scrape in pubmed


## Define Twitter related methods

In [14]:

#@title  Have borrowed these developer password keys from another person in another wipro team, we need to change it to a new keys 
TWITTER_KEY = 'QIqgjITOfksfMW4lRLDacQ' 
TWITTER_SECRET_KEY = 'R8x0xN9iSKXGNxUtGKA2hgnlIhh5INZIOdgEfxzk' 

import tweepy
from tweepy import OAuthHandler
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

tweepyapi = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not tweepyapi):
    print ("Can't Authenticate")
    sys.exit(-1)

twitterSearchquery = "  is good"
tweets = tweepyapi.search(twitterSearchquery)

print(tweets)


[Status(_api=<tweepy.api.API object at 0x7fdf854cc470>, _json={'created_at': 'Thu Aug 20 07:13:40 +0000 2020', 'id': 1296344634302423043, 'id_str': '1296344634302423043', 'text': "Good Vibrations is the Citizen Kane of music production and everyone gets tired of hearing about it but it's all true.", 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 88074587, 'id_str': '88074587', 'name': 'Eliot', 'screen_name': 'EliotETC', 'location': '', 'description': 'BLACK LIVES MATTER. I make videos. @InternetTodayTV w/ @RickyFTW. Pic by @Mikes_notsane. He/Him', 'url': 'https://t.co/sos8ghNA4d', 'entities': {'url': {'urls': [{'ur

In [15]:
def getlistofUniquetweets(searchQuery, maxTweets , unique_list):

  
  Filter_Retweets = True  

  tweetsPerQry = maxTweets  # this is the max the API permits
  tweet_lst = []

  if Filter_Retweets:
    searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

  # If results from a specific ID onwards are reqd, set since_id to that ID.
  # else default to no lower limit, go as far back as API allows
  sinceId = None

  # If results only below a specific ID are, set max_id to that ID.
  # else default to no upper limit, start from the most recent tweet matching the search query.
  max_id = -10000000000

  tweetCount = 0
  print("Downloading max {0} tweets".format(maxTweets))
  while tweetCount < maxTweets:
      try:
          if (max_id <= 0):
              if (not sinceId):
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang="en")
              else:
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',
                                          lang="en", since_id=sinceId)
          else:
              if (not sinceId):
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',
                                          lang="en", max_id=str(max_id - 1))
              else:
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',
                                          lang="en", max_id=str(max_id - 1),
                                          since_id=sinceId)
          if not new_tweets:
              print("No more tweets found")
              break
          for tweet in new_tweets:
            if hasattr(tweet, 'reply_count'):
              reply_count = tweet.reply_count
            else:
              reply_count = 0
            if hasattr(tweet, 'retweeted'):
              retweeted = tweet.retweeted
            else:
              retweeted = "NA"
              
            # fixup search query to get topic
            topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
            
            # fixup date
            tweetDate = tweet.created_at.date()
            
            ausername = tweet.user.screen_name
            
            if ausername not in unique_list: 
              unique_list.append(ausername) 

            tweet_lst.append([ tweetDate, topic, 
                        tweet.id, tweet.user.screen_name, tweet.user.name, tweet.full_text, tweet.favorite_count, 
                        reply_count, tweet.retweet_count, retweeted])

          tweetCount += len(new_tweets)
          print("Downloaded {0} tweets".format(tweetCount))
          max_id = new_tweets[-1].id
      except tweepy.TweepError as e:
          # Just exit if any error
          print("some error : " + str(e))
          break

  #clear_output()
  print("Downloaded {0} tweets".format(tweetCount))

  print("Identified {0} new candidate persons".format(len(unique_list)))
  #print("Identified {0} candidate persons".format(len(unique_candidates_list)))
  return unique_list




In [16]:
def getlistofUniqueContent(searchQuery, maxTweets , tweet_lst):
  unique_list = []

  orginalearchQuery = searchQuery + ''
  
  Filter_Retweets = True  

  tweetsPerQry = maxTweets  # this is the max the API permits
 
  if Filter_Retweets:
    searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

  # If results from a specific ID onwards are reqd, set since_id to that ID.
  # else default to no lower limit, go as far back as API allows
  sinceId = None

  # If results only below a specific ID are, set max_id to that ID.
  # else default to no upper limit, start from the most recent tweet matching the search query.
  max_id = -10000000000

  tweetCount = 0
  print("Downloading max {0} tweets".format(maxTweets))
  while tweetCount < maxTweets:
      try:
          if (max_id <= 0):
              if (not sinceId):
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang="en")
              else:
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',
                                          lang="en", since_id=sinceId)
          else:
              if (not sinceId):
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',
                                          lang="en", max_id=str(max_id - 1))
              else:
                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',
                                          lang="en", max_id=str(max_id - 1),
                                          since_id=sinceId)
          if not new_tweets:
              print("No more tweets found")
              break
          for tweet in new_tweets:
            if hasattr(tweet, 'reply_count'):
              reply_count = tweet.reply_count
            else:
              reply_count = 0
            if hasattr(tweet, 'retweeted'):
              retweeted = tweet.retweeted
            else:
              retweeted = "NA"
              
            # fixup search query to get topic
            topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
            
            # fixup date
            tweetDate = tweet.created_at.date()
            
            ausername = tweet.user.screen_name
            
            if ausername not in unique_candidates_list: 
              unique_list.append(ausername) 

            tweet_lst.append([ orginalearchQuery, searchQuery, tweetDate, topic, 
                        tweet.id, tweet.user.screen_name, tweet.user.name, tweet.full_text, tweet.favorite_count, 
                        reply_count, tweet.retweet_count, retweeted])

          tweetCount += len(new_tweets)
          print("Downloaded {0} tweets".format(tweetCount))
          max_id = new_tweets[-1].id
      except tweepy.TweepError as e:
          # Just exit if any error
          print("some error : " + str(e))
          break

 # clear_output()
  print("Downloaded {0} tweets".format(tweetCount))

  #print("Identified {0} new candidate persons".format(len(unique_list)))
  #print("Identified {0} candidate persons".format(len(unique_candidates_list)))
  return tweet_lst




In [17]:
def appendtolistofdoctors(HCPusersInfo, listofHCPs, datatoAppend):
  for auser  in listofHCPs:
    print(auser)
    test = api.lookup_users(screen_names=[auser])
    for user in test:
            HCPusersInfo.append({
                                u'KEYWORD':datatoAppend,
                                u'screen_name':user.screen_name,
                                u'name': user.name,
                                u'description':user.description,
                                u'followers_count':user.followers_count,
                                u'statuses_count': user.statuses_count,
                                u'URL': user.url     
                                })
  return HCPusersInfo

In [18]:
def get_tweets_AllofthemByUserName(username, result_limit): 
          

  
        # 1 tweets to be extracted 
        number_of_tweets=1
        tweets = api.user_timeline(screen_name=username, tweet_mode='extended'  , count=result_limit) 

      #  for atweet in tweets:
        #  print( atweet )
         # print( atweet.full_text)

   
        # Empty Array 
      #  tmp=[]  
  
        # create array of tweet information: username,  
        # tweet id, date/time, text 
      #  tweets_for_csv = [tweet.full_text for tweet in tweets] # CSV file created  
        #for j in tweets_for_csv: 
  
            # Appending tweets to the empty array tmp 
          #  tmp.append(j)        
  
        # Printing the tweets 
       # print(tmp) 
        return tweets


## Utils

In [19]:
import copy

In [20]:
def getOnlyDoctorsFromThisList(unique_candidates_list1, frequentKeywordsList):
  unique_candidates_listOnlyDoctors = []
  for aUser in unique_candidates_list1:
    countofmedicalVocabUttered = 0
    tweetsByaUser = get_tweets_AllofthemByUserName(aUser, 100)
    for asingletweet in tweetsByaUser:
      countofmedicalVocabUttered = countofmedicalVocabUttered + countofMedicalWordsInthisSentance(asingletweet.full_text,  frequentKeywordsList )
    if (countofmedicalVocabUttered > 5 ):
      unique_candidates_listOnlyDoctors.append(aUser)
  return unique_candidates_listOnlyDoctors

def countofMedicalWordsInthisSentance(mysentance, ListofMedicalVocab):
  mycountofmedicalwords = 0
  for aMedicalWord in ListofMedicalVocab:
    if aMedicalWord in mysentance:
      mycountofmedicalwords = mycountofmedicalwords + 1
  return mycountofmedicalwords

In [21]:
def getHCPandThierDistributionofInterests(unique_candidates_list2, frequentKeywordsList, HCPandKeywordsArrayofDict, KEYWORDSEARCHEDFORInTwitter ):
  for auser in unique_candidates_list2:
    A100tweetsByaUser = get_tweets_AllofthemByUserName(auser, 100)
    akeyworddistributiondictionaryforanuser = {}
    print(auser)
    userinfofromtwitterResults = api.lookup_users(screen_names=[auser])
    userinfofromtwitter = userinfofromtwitterResults[0]
    anewdictionary = dict( screenname= auser,  
                          name =  userinfofromtwitter.name, 
                          description =  userinfofromtwitter.description, 
                          followers_count =    userinfofromtwitter.followers_count,
                          statuses_count = userinfofromtwitter.statuses_count, 
                          URL = userinfofromtwitter.url 
                          )
    for akeyword in frequentKeywordsList:
      mycountofkeywordInSetsofTweets = countofkeywordInSetsofSentances( A100tweetsByaUser, akeyword)
      anewdictionary[akeyword] = mycountofkeywordInSetsofTweets    
    anewdictionary["Tweet"] = A100tweetsByaUser[0].full_text    
    HCPandKeywordsArrayofDict.append( anewdictionary )
    return HCPandKeywordsArrayofDict







In [22]:
def normalize(frequentKeywords, HCPandKeywordsArrayofDict):
  frequentKeywordsTotalSum = copy.deepcopy (frequentKeywords)
  for akeyword1 in frequentKeywordsTotalSum.keys():
    frequentKeywordsTotalSum[akeyword1] = 0

  for arow in HCPandKeywordsArrayofDict:
    for akeyword in frequentKeywords.keys() :
      frequentKeywordsTotalSum[akeyword]  += arow[akeyword]

  for arow in HCPandKeywordsArrayofDict:
    for akeyword in frequentKeywords.keys() :
      if frequentKeywordsTotalSum[akeyword].any() != 0:
        arow[akeyword] = arow[akeyword] / frequentKeywordsTotalSum[akeyword] 
  return HCPandKeywordsArrayofDict

In [23]:
def countofkeywordInSetsofSentances ( A100tweetsByaUser, aword):
  myanscounter = 0
  for atweet in A100tweetsByaUser:
    mysentance = atweet.full_text  
    if aword in mysentance:
      myanscounter =  myanscounter + 1
  return myanscounter

## Distrbution from normal users

In [24]:
def getRandomInterst(list_of_content, frequentKeywordsList ):

    

  anewdictionary = dict()

    

  for akeyword in frequentKeywordsList:

    mycountofkeywordInSetsofTweets = countofkeywordInSetsofSentances( list_of_content, akeyword)

    anewdictionary[akeyword] = mycountofkeywordInSetsofTweets    

 

  return anewdictionary

In [25]:
def getRandomTweets(maxTweets):

 

  

  Filter_Retweets = True  

 

  tweetsPerQry = maxTweets  # this is the max the API permits

  tweet_lst = []

 

  if Filter_Retweets:

    searchQuery = ' -filter:retweets'  # to exclude retweets

 

  # If results from a specific ID onwards are reqd, set since_id to that ID.

  # else default to no lower limit, go as far back as API allows

  sinceId = None

 

  # If results only below a specific ID are, set max_id to that ID.

  # else default to no upper limit, start from the most recent tweet matching the search query.

  max_id = -10000000000

 

  tweetCount = 0

  print("Downloading max {0} tweets".format(maxTweets))

  while tweetCount < maxTweets:

      try:

          if (max_id <= 0):

              if (not sinceId):

                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang="en")

              else:

                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',

                                          lang="en", since_id=sinceId)

          else:

              if (not sinceId):

                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',

                                          lang="en", max_id=str(max_id - 1))

              else:

                  new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended',

                                          lang="en", max_id=str(max_id - 1),

                                          since_id=sinceId)

          if not new_tweets:

              print("No more tweets found")

              break

          for tweet in new_tweets:

            if hasattr(tweet, 'reply_count'):

              reply_count = tweet.reply_count

            else:

              reply_count = 0

            if hasattr(tweet, 'retweeted'):

              retweeted = tweet.retweeted

            else:

              retweeted = "NA"

              

            # fixup date

            tweetDate = tweet.created_at.date()

            

            tweet_lst.append(tweet)

 

          tweetCount += len(new_tweets)

          #print("Downloaded {0} tweets".format(tweetCount))

          max_id = new_tweets[-1].id

      except tweepy.TweepError as e:

          # Just exit if any error

          print("some error : " + str(e))

          break

 

 # clear_output()

  print("Downloaded {0} tweets".format(tweetCount))

 

  return tweet_lst



In [26]:
#TODO Add 
#tweet_cnt = getRandomTweets(100000)

#getRandomInterst(tweet_cnt, frequentKeywordsListFiltered)



## List of Dates in pubmed

In [27]:
!pip install python-dateutil

In [28]:
from datetime import date

import dateutil

def get_ListofDatesAlsoGetaDictionary(numberofmonths, timespaninmonths, mydictionaryofMeSHqueriesCONTEXTDate):
  today = date.today()
  list_of_dates = []
  for i in range(numberofmonths): 
    d2 = today - dateutil.relativedelta.relativedelta(months=timespaninmonths)
    querystr = "(" + date.strftime(d2, format = '%Y/%m/%d') + "[Date - Create] : " + date.strftime(today, format = '%Y/%m/%d') + "[Date - Create] )"
    today = d2
    list_of_dates.append(querystr)
    mydictionaryofMeSHqueriesCONTEXTDate[querystr] = today
  return list_of_dates, mydictionaryofMeSHqueriesCONTEXTDate

##Generic function to look for most frequent KEY PHRASES using BioBERT

In [29]:
def CreateGenericNEW( focusColoum  , maxResults, query):
  articleInfo = []
  articleList = []

  if True:

    results = pubmed.query(query, max_results=maxResults)
    print("CreateGenericNEW" + query )
#    MesHTermsArray = mydictionaryofMeSHqueries[query]
    time.sleep(5)
    

    for article in results:
      # Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
      # We need to convert it to dictionary with available function
        articleDict = article.toDict()
        articleList.append(articleDict)

      # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
    for article in articleList:
        #print(article)
      #Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
        pubmedId = article['pubmed_id'].partition('\n')[0]
          # Append article info to dictionary 
        # articleInfo.append({u'pubmed_id':pubmedId,                        u'title':article['title'],                        u'keywords':article['keywords'],                       u'journal':article['journal'],})

        if not (focusColoum=='title' or focusColoum=='abstract' or focusColoum=='keyword'):
          print("ERROR in INPUT.. focusColoum must be title or abstract or keyword")

        if(focusColoum == 'keyword' ):
          if (len(article['keywords'])) > 0:
            for aword in article['keywords']:
              articleInfo.append({u'query':query,
                                  u'keywords':article['keywords'],
                                  u'KEYWORD':aword,
                                  u'authors':article['authors'],
                                  u'title':article['title'],
                                  u'abstract':article['abstract'],
                                  u'journal':article['journal'],
                                  u'doi':article['doi'],
                                  u'publication_date':article['publication_date'],
                                  u'authors':article['authors'],
                                  })

        if (focusColoum=='title' or focusColoum=='abstract'):
          if ((len(article['keywords']))  > 0) and ( article['abstract'] is not None):
            listofBioBERTentities = getNERBioBERT ( article[focusColoum])
            for aBioEntity in listofBioBERTentities:
              articleInfo.append({u'query':query,
                                  u'keywords':article['keywords'],
                                  u'KEYWORD': aBioEntity,
                                  u'title':article['title'],
                                  u'abstract':article['abstract'],
                                  u'journal':article['journal'],
                                  u'doi':article['doi'],
                                  u'publication_date':article['publication_date'],
                                  u'authors':article['authors'],                                     
                                  })
      
    # Generate Pandas DataFrame from list of dictionaries
  articlesPD = pd.DataFrame.from_dict(articleInfo)
    #export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

  articlesPD.info()

  #Print first 10 rows of dataframe
  articlesPD.head(10)
  articlesPD.tail(10)
  return articlesPD

In [30]:
def CreateGeneric( focusColoum  , maxResults):
  articleInfo = []
  articleList = []

  for query in ListofMeSHqueries:

    results = pubmed.query(query, max_results=maxResults)
    print(query)
    MesHTermsArray = mydictionaryofMeSHqueries[query]
    time.sleep(5)
    

    for article in results:
      # Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
      # We need to convert it to dictionary with available function
        articleDict = article.toDict()
        articleList.append(articleDict)

      # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
    for article in articleList:
      #Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
        pubmedId = article['pubmed_id'].partition('\n')[0]
          # Append article info to dictionary 
        # articleInfo.append({u'pubmed_id':pubmedId,                        u'title':article['title'],                        u'keywords':article['keywords'],                       u'journal':article['journal'],})

        if not (focusColoum=='title' or focusColoum=='abstract' or focusColoum=='keyword'):
          print("ERROR in INPUT.. focusColoum must be title or abstract or keyword")

        if(focusColoum == 'keyword' ):
          if (len(article['keywords'])) > 0:
            for aword in article['keywords']:
              articleInfo.append({u'query':query,
                                  u'MeSHTerm1': MesHTermsArray[0],
                                  u'keywords':article['keywords'],
                                  u'KEYWORD':aword,
                                  u'authors':article['authors'],
                                  u'title':article['title'],
                                  u'abstract':article['abstract'],
                                  u'journal':article['journal'],
                                  u'doi':article['doi'],
                                  u'publication_date':article['publication_date'],
                                  u'authors':article['authors'],
                                  })

        if (focusColoum=='title' or focusColoum=='abstract'):
          if ((len(article['keywords']))  > 0) and ( article['abstract'] is not None):
            listofBioBERTentities = getNERBioBERT ( article[focusColoum])
            for aBioEntity in listofBioBERTentities:
              articleInfo.append({u'query':query,
                                  u'MeSHTerm1': MesHTermsArray[0],
                                  u'keywords':article['keywords'],
                                  u'BioBERTKEYPHRASE': aBioEntity,
                                  u'title':article['title'],
                                  u'abstract':article['abstract'],
                                  u'journal':article['journal'],
                                  u'doi':article['doi'],
                                  u'publication_date':article['publication_date'],
                                  u'authors':article['authors'],                                     
                                  })
      
    # Generate Pandas DataFrame from list of dictionaries
  articlesPD = pd.DataFrame.from_dict(articleInfo)
    #export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

  articlesPD.info()

  #Print first 10 rows of dataframe
  articlesPD.head(10)
  articlesPD.tail(10)
  return articlesPD

##  Sentiment analysis using a ensemble of BERT and BioClinical BERT (ideally use BioClinical model if the content is clinical )

In [31]:
BERTsentimentanalyzer = pipeline('sentiment-analysis', model="simonlevine/Bio_ClinicalBERT-4096",  tokenizer="simonlevine/Bio_ClinicalBERT-4096")

#BERTsentimentanalyzer = pipeline('sentiment-analysis')
#TODO : Choice of BioBERT vs BERT for sentiment analysis or an attention based ensemble 

Some weights of the model checkpoint at simonlevine/Bio_ClinicalBERT-4096 were not used when initializing BertForSequenceClassification: ['encoder.layer.0.attention.self.query_global.weight', 'encoder.layer.0.attention.self.query_global.bias', 'encoder.layer.0.attention.self.key_global.weight', 'encoder.layer.0.attention.self.key_global.bias', 'encoder.layer.0.attention.self.value_global.weight', 'encoder.layer.0.attention.self.value_global.bias', 'encoder.layer.1.attention.self.query_global.weight', 'encoder.layer.1.attention.self.query_global.bias', 'encoder.layer.1.attention.self.key_global.weight', 'encoder.layer.1.attention.self.key_global.bias', 'encoder.layer.1.attention.self.value_global.weight', 'encoder.layer.1.attention.self.value_global.bias', 'encoder.layer.2.attention.self.query_global.weight', 'encoder.layer.2.attention.self.query_global.bias', 'encoder.layer.2.attention.self.key_global.weight', 'encoder.layer.2.attention.self.key_global.bias', 'encoder.layer.2.attention

In [32]:
BERTsentimentanalyzer ("i do NOT like it ")

[{'label': 'LABEL_1', 'score': 0.6679993867874146}]

In [33]:
def getSentiment(myinputsentance):
  ans = BERTsentimentanalyzer(myinputsentance)
  finalsentiment = 0
  if len(ans) > 0:
    if ans[0]['score'] > 0.9:
      if ans[0]['label'] == 'NEGATIVE' :
        finalsentiment = -1
      if ans[0]['label'] == 'POSITIVE' :
        finalsentiment = 1
  return finalsentiment


# Set the time for completion of the run

In [34]:
ISDEBUGMODE = 20
# set this value to 2 for 2 iterations of query. (10 iterations takes 5 minutes)
# set this value to 2000000 for all the iternations 

#    Step 1.   Based on BU end user needs, Let the BU define the scope of experiment here..

Medical Standard Heading MeSH based query strategy 
https://meshb-prev.nlm.nih.gov/treeView

### Formulate MeSH queries based on user selection on the UI

### Display a MeSH query to BU end user and preview expected results  

# Define the input MeSH matrix 

### TODO: directly read off a excel sheet 

In [35]:
ListCONTEXTMeSH_1 = ['Infant, Low Birth Weight', 'Child, Preschool']
ListCONTEXTMeSH_2 = ['Food and Beverages', 'Proteins', 'Dietary Carbohydrates']
ListCONTEXTMeSH_3 = ['*', 'Hepatitis', 'Neonatal Sepsis', 'Apnea']


In [36]:
ListCONTEXTMeSH_1 = ['Infant, Low Birth Weight' ]
ListCONTEXTMeSH_2 = ['Food and Beverages']
ListCONTEXTMeSH_3 = ['*' ]



*   PubMedQueryTimeSpan  = 1 (month wise search)
*   PubMedQueryTimeSpan  = 12 (year wise search)





In [37]:
PubMedQueryNoofDateQueries = 10 #@param {type:"slider", min:1, max:100, step:1}
PubMedQueryTimeSpan = 1 #@param {type:"slider", min:1, max:12, step:1}


In [38]:
ListofdatewisequeryTemp, mydictionaryofMeSHqueriesDatesTemp = get_ListofDatesAlsoGetaDictionary(PubMedQueryNoofDateQueries, PubMedQueryTimeSpan, {} )
ListofdatewisequeryTemp


['(2020/07/20[Date - Create] : 2020/08/20[Date - Create] )',
 '(2020/06/20[Date - Create] : 2020/07/20[Date - Create] )',
 '(2020/05/20[Date - Create] : 2020/06/20[Date - Create] )',
 '(2020/04/20[Date - Create] : 2020/05/20[Date - Create] )',
 '(2020/03/20[Date - Create] : 2020/04/20[Date - Create] )',
 '(2020/02/20[Date - Create] : 2020/03/20[Date - Create] )',
 '(2020/01/20[Date - Create] : 2020/02/20[Date - Create] )',
 '(2019/12/20[Date - Create] : 2020/01/20[Date - Create] )',
 '(2019/11/20[Date - Create] : 2019/12/20[Date - Create] )',
 '(2019/10/20[Date - Create] : 2019/11/20[Date - Create] )']

In [39]:
import copy

In [40]:
from IPython.display import display
from IPython.display import clear_output

In [41]:
ListofMeSHqueries = []
mydictionaryofMeSHqueriesCONTEXTMeSH_1 = {}
mydictionaryofMeSHqueriesCONTEXTMeSH_2 = {}
mydictionaryofMeSHqueriesCONTEXTMeSH_3 = {}
mydictionaryofMeSHqueriesCONTEXTDates = {}

ListofdatewisequeryTemp, mydictionaryofMeSHqueriesDatesTemp = get_ListofDatesAlsoGetaDictionary(PubMedQueryNoofDateQueries, PubMedQueryTimeSpan, mydictionaryofMeSHqueriesCONTEXTDates)
 
for CONTEXTMeSH_1 in ListCONTEXTMeSH_1:
  CONTEXTMeSH_1temp = copy.copy(CONTEXTMeSH_1)
  for CONTEXTMeSH_2 in ListCONTEXTMeSH_2:
    CONTEXTMeSH_2temp = copy.copy(CONTEXTMeSH_2)   
    for CONTEXTMeSH_3 in ListCONTEXTMeSH_3:
      CONTEXTMeSH_3temp = copy.copy(CONTEXTMeSH_3) 
      for CONTEXTdatequery in ListofdatewisequeryTemp:

        if CONTEXTMeSH_2 == '*':
          CONTEXTMeSH_2 =   copy.copy(CONTEXTMeSH_1)
        if CONTEXTMeSH_3 == '*':
          CONTEXTMeSH_3 = copy.copy(CONTEXTMeSH_1)        
        query = "(" +  CONTEXTMeSH_1 + "[MeSH Major Topic])"+" AND " + "(" +  CONTEXTMeSH_2 + "[MeSH Major Topic])"+" AND " + "(" +  CONTEXTMeSH_3 + "[MeSH Major Topic])"             
        query = query + " AND " + CONTEXTdatequery

        ListofMeSHqueries.append( copy.copy(query)+'')

      # count = scrape_save_count(query)
      # time.sleep(1)

        mydictionaryofMeSHqueriesCONTEXTMeSH_1[query] = CONTEXTMeSH_1temp
        mydictionaryofMeSHqueriesCONTEXTMeSH_2[query] = CONTEXTMeSH_2temp
        mydictionaryofMeSHqueriesCONTEXTMeSH_3[query] = CONTEXTMeSH_3temp     
        mydictionaryofMeSHqueriesCONTEXTDates[query] =  mydictionaryofMeSHqueriesDatesTemp[CONTEXTdatequery]
        print(  '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query]) + "\t" + str( mydictionaryofMeSHqueriesCONTEXTDates[query])+ "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"  + str(query) )      



	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	2020-07-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/07/20[Date - Create] : 2020/08/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	2020-06-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/06/20[Date - Create] : 2020/07/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	2020-05-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/05/20[Date - Create] : 2020/06/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	2020-04-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AN

In [42]:
ListofMeSHqueriesNonZero = []

In [43]:
for query in ListofMeSHqueries:
  count = scrape_save_count(query)
  if count != 0 :
    ListofMeSHqueriesNonZero.append (query)
  time.sleep(0.5)
  print( str(count) + '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query]) +  '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTDates[query]) +"\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"  + str(query)  )      


0	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-07-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/07/20[Date - Create] : 2020/08/20[Date - Create] )
1	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-06-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/06/20[Date - Create] : 2020/07/20[Date - Create] )
1	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-05-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/05/20[Date - Create] : 2020/06/20[Date - Create] )
2	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-04-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Bev

In [44]:
ListofMeSHqueries = ListofMeSHqueriesNonZero

In [45]:
for query in ListofMeSHqueries :
 # count = scrape_save_count(query)
  time.sleep(0.5)
  print(  '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query]) +  '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTDates[query]) +"\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"  + str(query)  )      


	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-06-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/06/20[Date - Create] : 2020/07/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-05-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/05/20[Date - Create] : 2020/06/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-04-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/04/20[Date - Create] : 2020/05/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-01-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverag

In [46]:
for query in ListofMeSHqueries:
  print(   '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query]) +  '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTDates[query]) +"\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"  + str(query)  )      


	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-06-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/06/20[Date - Create] : 2020/07/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-05-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/05/20[Date - Create] : 2020/06/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-04-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverages[MeSH Major Topic]) AND (Infant, Low Birth Weight[MeSH Major Topic]) AND (2020/04/20[Date - Create] : 2020/05/20[Date - Create] )
	 	 : 	Infant, Low Birth Weight	Food and Beverages	*	 	 : 	2020-01-20																(Infant, Low Birth Weight[MeSH Major Topic]) AND (Food and Beverag

# Dataset 1 :  HCPs from twitter and frequenly used keywords


In [47]:
bioNER = getNERBioBERT_Init(4)

### Use the generic function but using KEYWORDs mentioned in article by the authors of the paper

In [48]:
NoofMaxArticlestoGetFromPubMedPerQuery = 30 #@param {type:"slider", min:10, max:500, step:1}

In [49]:
CONTEXTMeSH_1 = ''
CONTEXTMeSH_2 = ''
CONTEXTMeSH_3 = ''
articlesMASTERDataFrame  = None
debugconter = 0

for query in ListofMeSHqueries:
  print(query)
  CONTEXTMeSH_1 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query])
  CONTEXTMeSH_2 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query])
  CONTEXTMeSH_3 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query])
  CONTEXTdate = mydictionaryofMeSHqueriesCONTEXTDates[query]
 # articlesDataFrame = CreateGenericNEW('keyword', NoofMaxArticlestoGetFromPubMedPerQuery , query)
  articlesDataFrame = CreateGenericNEW('title', NoofMaxArticlestoGetFromPubMedPerQuery , query)
  articlesDataFrame['CONTEXTMeSH_1'] = CONTEXTMeSH_1
  articlesDataFrame['CONTEXTMeSH_2'] = CONTEXTMeSH_2
  articlesDataFrame['CONTEXTMeSH_3'] = CONTEXTMeSH_3
  articlesDataFrame['CONTEXTDate'] = CONTEXTdate


  if debugconter  == 0  :
    articlesMASTERDataFrame = copy.deepcopy(articlesDataFrame)
    print("initatied")
  else :
    temp =   pd.concat([articlesMASTERDataFrame, articlesDataFrame], ignore_index=True)
    articlesMASTERDataFrame = copy.deepcopy(temp)
    print("COPIED ") 
    print( articlesMASTERDataFrame.count(axis=0))
 
  print( str(count) + '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query]) +  "\t\t" +  str(mydictionaryofMeSHqueriesCONTEXTDates[query])   + "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"  + str(query) )      
  time.sleep(0.1)
  debugconter = debugconter + 1
  clear_output()
  if debugconter > ISDEBUGMODE + 12 :
   break



In [50]:
articlesMASTERDataFrame

,CONTEXTMeSH_1,CONTEXTMeSH_2,CONTEXTMeSH_3,CONTEXTDate,query,keywords,KEYWORD,title,abstract,journal,doi,publication_date,authors
0,"Infant, Low Birth Weight",Food and Beverages,*,2019-10-20,"(Infant, Low Birth Weight[MeSH Major Topic]) A...","[donor milk, genetics, inflammation, intestina...",exfoliated intestinal cells,Impact of Donor Human Milk in the Preterm Very...,Own mother's milk (OMM) is the optimal nutriti...,Nutrients,10.3390/nu11112677,2019-11-07,"[{'lastname': 'Parra-Llorca', 'firstname': 'An..."


In [51]:
articlesMASTERDataFrame.to_csv('pubmed_articles_byKeywords.csv')

In [52]:
files.download('pubmed_articles_byKeywords.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
sfsfsfsfs


NameError: ignored

## Find keyword distribution 

In [ ]:
STOPNonMedicalWords = ['Risk factors', 'follow-up']

## Keyword distribution with filyers by Context query such as MesH combination and date

In [ ]:
CONTEXTMeSH_1 = ''
CONTEXTMeSH_2 = ''
CONTEXTMeSH_3 = ''
#articlesMASTERDataFrame  = None
frequentKeywordsMASTER = None
isFirstTime = True
debugconter = 0
 
for query in ListofMeSHqueries:
  print( 'ITERATION NUMBER = ' + str(debugconter) )
  print(query)
  CONTEXTMeSH_1 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query])
  CONTEXTMeSH_2 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query])
  CONTEXTMeSH_3 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query])
  CONTEXTdate = mydictionaryofMeSHqueriesCONTEXTDates[query]  
  articlesDataFrame = CreateGenericNEW('keyword', 30 , query)

  if( len(articlesDataFrame.index) > 0 ):
    if 'KEYWORD' in articlesDataFrame.columns: 
      frequentKeywords = articlesDataFrame['KEYWORD'].value_counts()
      frequentKeywords1 = frequentKeywords[0:10]
      frequentKeywordsList = frequentKeywords1.keys ()
      frequentKeywordsListFiltered = []

      for aword in frequentKeywordsList:
        if not ( aword in STOPNonMedicalWords):
          frequentKeywordsListFiltered.append(aword)

      frequentKeywords = frequentKeywords.to_frame()
      frequentKeywords['CONTEXTMeSH_1'] = CONTEXTMeSH_1
      frequentKeywords['CONTEXTMeSH_2'] = CONTEXTMeSH_2
      frequentKeywords['CONTEXTMeSH_3'] = CONTEXTMeSH_3
      frequentKeywords['CONTEXTDate'] = CONTEXTdate
      

      #frequentKeywords.to_csv('Frequent_Keywords_ForaMesHQuery.csv')
      if isFirstTime  == True  :
        frequentKeywordsMASTER = copy.deepcopy(frequentKeywords)
        isFirstTime = False
        print("initatied")
      else :
        temp =   pd.concat([frequentKeywordsMASTER, frequentKeywords] )
#        temp =   pd.concat([frequentKeywordsMASTER, frequentKeywords], ignore_index=True)
        frequentKeywordsMASTER = copy.deepcopy(temp)
        print("COPIED ") 
        print( frequentKeywordsMASTER.count())


  print( str(count) + '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query]) + "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"  + str(query) )      
  print( 'ITERATION NUMBER = ' + str(debugconter) )
  time.sleep(0.1)
  debugconter = debugconter + 1
  clear_output()
  if debugconter > ISDEBUGMODE + 10 :
   break


In [ ]:
frequentKeywordsMASTER.to_csv('Frequent_Keywords_ForaMesHQueryWithDates.csv')

In [ ]:
files.download('Frequent_Keywords_ForaMesHQueryWithDates.csv')

In [ ]:
  frequentKeywordsMASTER

In [ ]:
STOPNonMedicalWords = ['Risk factors', 'follow-up']

 

## Get HCPs to Keyword distribution dataset for visuvalization

In [ ]:
def addHCPs(frequentKeywordsListFiltered, frequentKeywordsList2, HCPandKeywordsArrayofDict):
  if HCPandKeywordsArrayofDict is None:
    HCPandKeywordsArrayofDict = []
  for aKeyword in frequentKeywordsListFiltered:    
    unique_candidates_list = [] 
    unique_candidates_list = getlistofUniquetweets(aKeyword, 25, unique_candidates_list)
    HCPandKeywordsArrayofDict = getHCPandThierDistributionofInterests( unique_candidates_list,  frequentKeywordsList2.keys() , HCPandKeywordsArrayofDict, aKeyword  )
    print( (pd.DataFrame.from_dict(HCPandKeywordsArrayofDict)) )
    if HCPandKeywordsArrayofDict is None:
      HCPandKeywordsArrayofDict = []
      continue
    HCPandKeywordsArrayofDictNormalized = normalize(frequentKeywordsList2, HCPandKeywordsArrayofDict)
    #HCPandKeywordsArrayofDictNormalized = HCPandKeywordsArrayofDict
    HCPandKeywordsArrayofDictNormalizedSelected = selectHCPs(HCPandKeywordsArrayofDictNormalized , frequentKeywordsList2.keys()  )
  return HCPandKeywordsArrayofDictNormalizedSelected



In [ ]:
def selectHCPs(HCPandKeywordsArrayofDictNormalized, frequentKeywordsList2):
  for arow in HCPandKeywordsArrayofDictNormalized :
    diversityofexpertise = 0
    totalnumberofmedicalutterances = 0
    for akeyword in frequentKeywordsList2 :
      if arow[akeyword] != 0:
        diversityofexpertise += 1
        totalnumberofmedicalutterances += arow[akeyword]
    arow["Totalnumberofmedicalutterances"] = totalnumberofmedicalutterances
    arow["Diversityofexpertise"] = diversityofexpertise
    if diversityofexpertise > 4 and totalnumberofmedicalutterances > 0.3 :
      arow["Algo1"] = 1
    else :
      arow["Algo1"] = 0
    if diversityofexpertise > 3 and totalnumberofmedicalutterances > 0.1 :
      arow["Algo2"] = 1
    else :
      arow["Algo2"] = 0
    if  totalnumberofmedicalutterances > 0.1 :
      arow["Algo3"] = 1
    else :
      arow["Algo3"] = 0 
    return HCPandKeywordsArrayofDictNormalized

In [ ]:
api = tweepyapi
#define a dictionary to hold all the doctors info
#HCPusersInfo = []
#unique_candidates_list = [] 
 

In [ ]:
CONTEXTMeSH_1 = ''
CONTEXTMeSH_2 = ''
CONTEXTMeSH_3 = ''
#articlesMASTERDataFrame  = None
HCPandKeywordsDFMASTER = None
isFirstTime = True
debugconter = 0
 
for query in ListofMeSHqueries:
  HCPandKeywordsArrayofDict = []
  print( 'ITERATION NUMBER = ' + str(debugconter) )
  print(query)
  CONTEXTMeSH_1 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query])
  CONTEXTMeSH_2 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query])
  CONTEXTMeSH_3 = str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query])
  CONTEXTdate = mydictionaryofMeSHqueriesCONTEXTDates[query]  
  articlesDataFrame = CreateGenericNEW('keyword', 30 , query)

  if( len(articlesDataFrame.index) > 0 ):
    if 'KEYWORD' in articlesDataFrame.columns: 
      frequentKeywordsSeries = articlesDataFrame['KEYWORD'].value_counts()
      frequentKeywords1 = frequentKeywordsSeries[0:10]
      frequentKeywordsList = frequentKeywords1.keys ()
      frequentKeywordsListFiltered = []

      for aword in frequentKeywordsList:
        if not ( aword in STOPNonMedicalWords):
          frequentKeywordsListFiltered.append(aword)

      HCPandKeywordsArrayofDictNormalizedSelected = addHCPs(frequentKeywordsListFiltered, frequentKeywordsSeries, HCPandKeywordsArrayofDict)      

     # HCPandKeywordsDF = HCPandKeywordsArrayofDictNormalizedSelected.to_frame()
      HCPandKeywordsDF = pd.DataFrame.from_dict(HCPandKeywordsArrayofDictNormalizedSelected)

      HCPandKeywordsDF['CONTEXTMeSH_1'] = CONTEXTMeSH_1
      HCPandKeywordsDF['CONTEXTMeSH_2'] = CONTEXTMeSH_2
      HCPandKeywordsDF['CONTEXTMeSH_3'] = CONTEXTMeSH_3
      HCPandKeywordsDF['CONTEXTDate'] = CONTEXTdate

      #frequentKeywords.to_csv('Frequent_Keywords_ForaMesHQuery.csv')
      if isFirstTime  == True  :
        HCPandKeywordsDFMASTER = copy.deepcopy(HCPandKeywordsDF)
        isFirstTime = False
        print("initatied")
      else :
        temp =   pd.concat([HCPandKeywordsDFMASTER, HCPandKeywordsDF] )
#        temp =   pd.concat([frequentKeywordsMASTER, frequentKeywords], ignore_index=True)
        HCPandKeywordsDFMASTER = copy.deepcopy(temp)
        print("COPIED ") 
        print( HCPandKeywordsDFMASTER.count())


  print( str(count) + '\t \t : \t' + str(mydictionaryofMeSHqueriesCONTEXTMeSH_1[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_3[query]) + "\t" + str(mydictionaryofMeSHqueriesCONTEXTMeSH_2[query]) + "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"  + str(query) )      
  print( 'ITERATION NUMBER = ' + str(debugconter) )
  time.sleep(0.1)
  debugconter = debugconter + 1
  clear_output()
  if debugconter > ISDEBUGMODE + 12 :
   break


In [ ]:
HCPandKeywordsDFMASTER = HCPandKeywordsDFMASTER.replace(np.nan,0)


In [ ]:
HCPandKeywordsDFMASTER.to_csv('HCPandKeywordsDFMASTER.csv')

In [ ]:
files.download('HCPandKeywordsDFMASTER.csv')

In [ ]:
HCPandKeywordsDFMASTER

# run until here

In [ ]:
ENDHERE

In [ ]:
frequentKeywordsListFiltered

In [ ]:
frequentKeywordsListFiltered

In [ ]:
frequentKeywordsListFiltered

In [ ]:
frequentKeywordsSeries 

In [ ]:
frequentKeywordsSeries.keys()

In [ ]:
HCPandKeywordsArrayofDict

In [ ]:
HCPandKeywordsArrayofDictNormalizedSelected = addHCPs(frequentKeywordsListFiltered, frequentKeywordsSeries, HCPandKeywordsArrayofDict)

In [ ]:
HCPandKeywordsArrayofDictNormalizedSelected

In [ ]:
HCPandKeywordsArrayofDictPD = pd.DataFrame.from_dict(HCPandKeywordsArrayofDictNormalizedSelected)
HCPandKeywordsArrayofDictPD

In [ ]:
 frequentKeywords.keys()

In [ ]:
for aKeyword in frequentKeywordsListFiltered:
    
    unique_candidates_list = [] 
    unique_candidates_list = getlistofUniquetweets(aKeyword, 25, unique_candidates_list)
    #print(unique_candidates_list, frequentKeywords.key() )
    #TODO Insert a function here to remove candidates who are NOT really professional doctors
    unique_candidates_listOnlyHCPs = getOnlyDoctorsFromThisList(unique_candidates_list, frequentKeywords.keys()   )
    getHCPandThierDistributionofInterests( unique_candidates_list,  frequentKeywords.keys() , HCPandKeywordsArrayofDict, aKeyword  )
    #print('Identified HCPs based on there tweets are ' + str( len(unique_candidates_listOnlyHCPs) ))
    #HCPusersInfo = appendtolistofdoctors(HCPusersInfo , unique_candidates_listOnlyHCPs, aKeyword)
     


In [ ]:
HCPandKeywordsArrayofDict

In [ ]:
HCPandKeywordsArrayofDictNormalized = normalize(frequentKeywords, HCPandKeywordsArrayofDict)

In [ ]:
for arow in HCPandKeywordsArrayofDictNormalized :
  diversityofexpertise = 0
  totalnumberofmedicalutterances = 0
  for akeyword in frequentKeywords.keys() :
    if arow[akeyword] != 0:
      diversityofexpertise += 1
      totalnumberofmedicalutterances += arow[akeyword]
  arow["Totalnumberofmedicalutterances"] = totalnumberofmedicalutterances
  arow["Diversityofexpertise"] = diversityofexpertise
  if diversityofexpertise > 4 and totalnumberofmedicalutterances > 0.3 :
    arow["Algo1"] = 1
  else :
    arow["Algo1"] = 0
  if diversityofexpertise > 3 and totalnumberofmedicalutterances > 0.1 :
    arow["Algo2"] = 1
  else :
    arow["Algo2"] = 0
  if  totalnumberofmedicalutterances > 0.1 :
    arow["Algo3"] = 1
  else :
    arow["Algo3"] = 0 

In [ ]:
HCPandKeywordsArrayofDictPD = pd.DataFrame.from_dict(HCPandKeywordsArrayofDictNormalized)

HCPandKeywordsArrayofDictPD.to_csv('HCPandKeywordsArrayofDictPD.csv')

In [ ]:
HCPandKeywordsArrayofDictPD

In [ ]:
for aKeyword in frequentKeywordsListFiltered:
    
    unique_candidates_list = [] 
    unique_candidates_list = getlistofUniquetweets(aKeyword, 25, unique_candidates_list)
    #print(unique_candidates_list, frequentKeywords.key() )
    #TODO Insert a function here to remove candidates who are NOT really professional doctors
    unique_candidates_listOnlyHCPs = getOnlyDoctorsFromThisList(unique_candidates_list, frequentKeywords.keys()   )
    getHCPandThierDistributionofInterests( unique_candidates_list,  frequentKeywords.keys() , HCPandKeywordsArrayofDict, aKeyword  )
    print('Identified HCPs based on there tweets are ' + str( len(unique_candidates_listOnlyHCPs) ))
    HCPusersInfo = appendtolistofdoctors(HCPusersInfo , unique_candidates_listOnlyHCPs, aKeyword)
     


In [ ]:
HCPandKeywordsArrayofDictNormalized = normalize(frequentKeywords, HCPandKeywordsArrayofDict)

In [ ]:
for arow in HCPandKeywordsArrayofDictNormalized :
  diversityofexpertise = 0
  totalnumberofmedicalutterances = 0
  for akeyword in frequentKeywords.keys() :
    if arow[akeyword] != 0:
      diversityofexpertise += 1
      totalnumberofmedicalutterances += arow[akeyword]
  arow["Totalnumberofmedicalutterances"] = totalnumberofmedicalutterances
  arow["Diversityofexpertise"] = diversityofexpertise
  if diversityofexpertise > 4 and totalnumberofmedicalutterances > 0.3 :
    arow["Algo1"] = 1
  else :
    arow["Algo1"] = 0
  if diversityofexpertise > 3 and totalnumberofmedicalutterances > 0.1 :
    arow["Algo2"] = 1
  else :
    arow["Algo2"] = 0
  if  totalnumberofmedicalutterances > 0.1 :
    arow["Algo3"] = 1
  else :
    arow["Algo3"] = 0 

In [ ]:
#HCPandKeywordsArrayofDictNormalized

In [ ]:
HCPandKeywordsArrayofDictPD = pd.DataFrame.from_dict(HCPandKeywordsArrayofDictNormalized)

In [ ]:
 HCPandKeywordsArrayofDictPD

In [ ]:
HCPandKeywordsArrayofDictPD.to_csv('HCPandKeywordsArrayofDictPD.csv')

In [ ]:
files.download('HCPandKeywordsArrayofDictPD.csv')

In [ ]:
frequentKeywords.to_csv('Frequent_Keywords_ForaMesHQuery.csv')

In [ ]:
files.download('Frequent_Keywords_ForaMesHQuery.csv')

In [ ]:
HCPusersInfoPD.to_csv( 'HCPsfromTwitterForTop10words.csv')

In [ ]:
HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)
HCPusersInfoPD.info()
HCPusersInfoPD

In [ ]:
files.download('HCPsfromTwitterForTop10words.csv')

## Find an answer to the question by an expert at *Nestle* Nutrition Institute  

In [ ]:
#bioQA = pipeline('question-answering', model="monologg/biobert_v1.1_pubmed", tokenizer="monologg/biobert_v1.1_pubmed")
bioQA = pipeline('question-answering', model="ktrapeznikov/biobert_v1.1_pubmed_squad_v2", tokenizer="ktrapeznikov/biobert_v1.1_pubmed_squad_v2")


In [ ]:
bioQA = pipeline('question-answering')

In [ ]:
bioQA(context='  infant nutrition products are good', question='What products are good?')

In [ ]:
answerQA = bioQA(context='  infant nutrition products are good .   HMO formula products is also good', question='What products are good?')

In [ ]:
answerQA

In [ ]:
len(answerQA)

In [ ]:
if len(answerQA) > 1:
  if float(answerQA["score"]) > 0.1 :
    answerQA["answer"]
    print(answerQA["answer"])

In [ ]:
BUendUserQuestion = "what is the significance?"

In [ ]:


for index, row in articlesDataFrame.iterrows():
    if index == 80:
      break
    contentabstract = row['abstract']
    try:
      answerQA = bioQA(context=contentabstract, question=BUendUserQuestion)
      if len(answerQA) > 1:
        if float(answerQA["score"]) > 0.01 :
          answerQA["answer"]
          print(answerQA["answer"] , contentabstract)
    except:
      print(' ')

In [ ]:
BUendUserQuestion = "whether isolated maternal hypothyroxinaemia (IMH) is associated with risks?"

In [ ]:


for index, row in articlesDataFrame.iterrows():
    if index == 80:
      break
    contentabstract = row['abstract']
    try:
      answerQA = bioQA(context=contentabstract, question=BUendUserQuestion)
      if len(answerQA) > 1:
        if float(answerQA["score"]) > 0.01 :
          answerQA["answer"]
          print(answerQA["answer"] )
          print(contentabstract)
    except:
      print(' ')

In [ ]:
BUendUserQuestion = "What is a preventable cause of perinatal mortality?"

In [ ]:


for index, row in articlesDataFrame.iterrows():
    if index == 80:
      break
    contentabstract = row['abstract']
    try:
      answerQA = bioQA(context=contentabstract, question=BUendUserQuestion)
      if len(answerQA) > 1:
        if float(answerQA["score"]) > 0.01 :
          answerQA["answer"]
          print(answerQA["score"] , ' \t\t:\t\t', answerQA["answer"] )
    except:
      print(' ')

In [ ]:
bioQA = pipeline('question-answering', model="ktrapeznikov/biobert_v1.1_pubmed_squad_v2", tokenizer="ktrapeznikov/biobert_v1.1_pubmed_squad_v2")


In [ ]:
stopehrere

### get tweet content as well

In [ ]:
tweetcontent = []

for aKeyword in frequentKeywordsListFiltered:
    
    tweetcontent = getlistofUniqueContent(aKeyword, 10, tweetcontent)
    #TODO Insert a function here to remove   NOT   professional content


In [ ]:
tweetcontentPD = pd.DataFrame.from_dict (tweetcontent )

In [ ]:
tweetcontentPD.to_csv('TweetContentbyKeyword.csv')

In [ ]:
files.download('TweetContentbyKeyword.csv')

In [ ]:
tweetcontentPD

In [ ]:
STOPHERE>...

## Use the generic function but now use NER from BioBERT for Key PHRASES  

In [ ]:
bioNER = getNERBioBERT_Init(2)

In [ ]:
articlesDataFrameBIO = CreateGeneric('title', 10)

In [ ]:
articlesDataFrameBIO.to_csv('pubmed_articles_byBIOBERTKeyPHRASE.csv')

In [ ]:
files.download('pubmed_articles_byBIOBERTKeyPHRASE.csv')

In [ ]:
frequentKeywords = articlesDataFrameBIO['BioBERTKEYPHRASE'].value_counts()
frequentKeywords1 = frequentKeywords[0:20]
frequentKeywordsList = frequentKeywords1.keys ()
STOPNonMedicalWords = ['Risk factors', 'follow-up', 'the', 'ma', 'impact', "'"]
frequentKeywordsListFiltered = []

for aword in frequentKeywordsList:
  if not ( aword in STOPNonMedicalWords):
    frequentKeywordsListFiltered.append(aword)

frequentKeywordsListFiltered

In [ ]:
frequentKeywords

In [ ]:
api = tweepyapi
#define a dictionary to hold all the doctors info
HCPusersInfo = []
unique_candidates_list = [] 

In [ ]:
for aKeyword in frequentKeywordsListFiltered:
    
    unique_candidates_list = [] 
    unique_candidates_list = getlistofUniquetweets(aKeyword, 10, unique_candidates_list)
    print(unique_candidates_list)
    #TODO Insert a function here to remove candidates who are NOT really professional doctors
    HCPusersInfo = appendtolistofdoctors(HCPusersInfo , unique_candidates_list, aKeyword)

#    HCPusersInfo = appendtoHCPInfo(unique_candidates_list, HCPusersInfo)
#HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)

In [ ]:
HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)
HCPusersInfoPD.info()
HCPusersInfoPD

In [ ]:
HCPusersInfoPD.to_csv( 'HCPsfromTwitterForTop10BIOpharase.csv')

In [ ]:
files.download('HCPsfromTwitterForTop10BIOpharase.csv')

In [ ]:
STOPHERE

In [ ]:
articleInfo = []
articleList = []

for query in ListofMeSHqueries:

  results = pubmed.query(query, max_results=30)
  print(query)
  MesHTermsArray = mydictionaryofMeSHqueries[query]
  time.sleep(5)
  

  for article in results:
    # Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
    # We need to convert it to dictionary with available function
      articleDict = article.toDict()
      articleList.append(articleDict)

    # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
  for article in articleList:
    #Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
      pubmedId = article['pubmed_id'].partition('\n')[0]
        # Append article info to dictionary 
      # articleInfo.append({u'pubmed_id':pubmedId,                        u'title':article['title'],                        u'keywords':article['keywords'],                       u'journal':article['journal'],})


      if (len(article['keywords'])) > 0:
        for aword in article['keywords']:
          articleInfo.append({u'query':query,
                              u'MeSHTerm1': MesHTermsArray[0],
                              u'keywords':article['keywords'],
                              u'KEYWORD':aword,
                              u'authors':article['keywords'],
                              u'title':article['title'],
                              u'abstract':article['abstract'],
                              u'journal':article['journal'],
                              u'doi':article['doi'],
                              u'publication_date':article['publication_date'],
                              u'authors':article['authors'],
                              })
    
  # Generate Pandas DataFrame from list of dictionaries
articlesPD = pd.DataFrame.from_dict(articleInfo)
  #export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

articlesPD.info()

#Print first 10 rows of dataframe
articlesPD.head(10)
articlesPD.tail(10)

In [ ]:
articlesPD.to_csv('pubmed_articles_byKEYWORD.csv')

In [ ]:
files.download('pubmed_articles_byKEYWORD.csv')

## Most frequently appearing keywords in this query

In [ ]:
frequentKeywords = articlesPD['KEYWORD'].value_counts()

### Take top 10 key phrases from BioBERT

In [ ]:
frequentKeywords1 = frequentKeywords[0:10]

In [ ]:
frequentKeywordsList = frequentKeywords1.keys ()

In [ ]:
frequentKeywordsList

In [ ]:
#Have a UI to filter off non-medical words
frequentKeywordsList

In [ ]:
STOPNonMedicalWords = ['Risk factors', 'follow-up']

frequentKeywordsListFiltered = []

for aword in frequentKeywordsList:
  if not ( aword in STOPNonMedicalWords):
    frequentKeywordsListFiltered.append(aword)

In [ ]:
frequentKeywordsListFiltered

In [ ]:
api = tweepyapi
#define a dictionary to hold all the doctors info
HCPusersInfo = []
unique_candidates_list = [] 

In [ ]:
for aKeyword in frequentKeywordsListFiltered:
    
    unique_candidates_list = [] 
    unique_candidates_list = getlistofUniquetweets(aKeyword, 10, unique_candidates_list)
    print(unique_candidates_list)
    #TODO Insert a function here to remove candidates who are NOT really professional doctors
    HCPusersInfo = appendtolistofdoctors(HCPusersInfo , unique_candidates_list, aKeyword)

#    HCPusersInfo = appendtoHCPInfo(unique_candidates_list, HCPusersInfo)
#HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)

In [ ]:
unique_candidates_list

In [ ]:
frequentKeywordsList = frequentKeywords.keys()

In [ ]:
frequentKeywordsList

In [ ]:
len(HCPusersInfo)

In [ ]:
HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)
HCPusersInfoPD.info()
HCPusersInfoPD

In [ ]:
HCPusersInfoPD.to_csv( 'HCPsfromTwitterForTop10words.csv')

In [ ]:
files.download('HCPsfromTwitterForTop10words.csv')

In [ ]:
END_HERE_FORNOW

# 2. Use BioBERT to get more medical keywords from the expert's article   

In [ ]:
bioNER = getNERBioBERT_Init(1)


In [ ]:
articlesDataFrame = CreateGeneric('title', 2)


## Most frequently appearing BioBERT phrases in Titles in this query

In [ ]:
frequentKeywordsBioBERT = articlesPD['BioBERT Entity'].value_counts()

In [ ]:
frequentKeywordsBioBERT

In [ ]:
frequentKeywordsBioBERTTop100 = frequentKeywordsBioBERT[0:100]

In [ ]:
frequentKeywordsBioBERTTop100[5:20]

In [ ]:
ListofTop10Phrases = frequentKeywordsBioBERTTop100.keys()

In [ ]:
ListofTop10Phrases

# Use Bio Clinical BERT to most frequented words

In [ ]:
bioNER = getNERBioBERT_Init(2)

In [ ]:
articleInfo = []
articleList = []

for query in ListofMeSHqueries:

  results = pubmed.query(query, max_results=3)
  print(query)
  MesHTermsArray = mydictionaryofMeSHqueries[query]
  time.sleep(5)
  

  for article in results:
    # Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
    # We need to convert it to dictionary with available function
      articleDict = article.toDict()
      articleList.append(articleDict)

    # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
  for article in articleList:
    #Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
      pubmedId = article['pubmed_id'].partition('\n')[0]
        # Append article info to dictionary 
      # articleInfo.append({u'pubmed_id':pubmedId,                        u'title':article['title'],                        u'keywords':article['keywords'],                       u'journal':article['journal'],})

      listofBioBERTentities = getNERBioBERT ( article['title'])
      if (len(article['keywords'])) > 0:
        for aBioEntity in listofBioBERTentities:
          articleInfo.append({u'query':query,
                              u'MeSHTerm1': MesHTermsArray[0],
                              u'keywords':article['keywords'],
                              u'Title':article['title'],
                              u'Bio Clinical BERT Entity': aBioEntity   
                              })
    
  # Generate Pandas DataFrame from list of dictionaries
articlesPD = pd.DataFrame.from_dict(articleInfo)
  #export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

articlesPD.info()

#Print first 10 rows of dataframe
articlesPD.head(10)
articlesPD.tail(10)

In [ ]:
frequentKeywordsBioClinicalBERT = articlesPD['Bio Clinical BERT Entity'].value_counts()

In [ ]:
frequentKeywordsBioClinicalBERTTop100 = frequentKeywordsBioClinicalBERT[0:100]

frequentKeywordsBioClinicalBERTTop100List = frequentKeywordsBioClinicalBERTTop100.keys()

In [ ]:
frequentKeywordsBioClinicalBERTTop100

In [ ]:
frequentKeywordsBioClinicalBERTTop100List

In [ ]:
bioNER = getNERBioBERT_Init(1)

In [ ]:
articlesDataFrame = CreateGeneric('title', 2)

#articlesDataFrame = CreateGeneric('abstract', 2)

articlesDataFrame

In [ ]:
frequentKeywordsBioClinicalBERTList = (articlesDataFrame['BioBERT Entity'].value_counts()) [0:100].keys()

In [ ]:
frequentKeywordsBioClinicalBERTList

In [ ]:
pdHCPs = getTwitterHCPs(frequentKeywordsBioClinicalBERTList)

In [ ]:
pdHCPs

In [ ]:
twitterSearchquery('SIRT1')

In [ ]:
articlesPD

In [ ]:
sgsgsgs

# 3. Go to twitter and look for people who are talking about these medical *topics*

In [ ]:
articleInfo = []
articleList = []

for query in ListofMeSHqueries:

  results = pubmed.query(query, max_results=3)
  print(query)
  MesHTermsArray = mydictionaryofMeSHqueries[query]
  time.sleep(5)
  

  for article in results:
    # Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
    # We need to convert it to dictionary with available function
      articleDict = article.toDict()
      articleList.append(articleDict)

    # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
  for article in articleList:
    #Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
      pubmedId = article['pubmed_id'].partition('\n')[0]
        # Append article info to dictionary 
      # articleInfo.append({u'pubmed_id':pubmedId,                        u'title':article['title'],                        u'keywords':article['keywords'],                       u'journal':article['journal'],})


      if (len(article['keywords'])) > 0:
        articleInfo.append({u'query':query,
                            u'MeSHTerm1': MesHTermsArray[0],
                            u'keywords':article['keywords'],
                            u'Title':article['title'],
                            u'BioBERT Entities': getBioEntitiesfromBioBERT ( article['title'] ),
                            u'tweets': tweepyapi.search( MesHTermsArray[0]) [0],
                            })
    
  # Generate Pandas DataFrame from list of dictionaries
articlesPD = pd.DataFrame.from_dict(articleInfo)
  #export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

articlesPD.info()

#Print first 10 rows of dataframe
articlesPD.head(10)
articlesPD.tail(10)

In [ ]:
api = tweepyapi
HCPusersInfo = []

In [ ]:
for aKeyword in frequentKeywordsList:
  unique_candidates_list = [] 
  unique_candidates_list = getlistofUniquetweets(aKeyword, 10, unique_candidates_list)
  HCPusersInfo = appendtoHCPInfo(unique_candidates_list, HCPusersInfo)


In [ ]:
def getTwitterHCPs(frequentKeywordsList):
  HCPusersInfo = []
  for aKeyword in frequentKeywordsList:
    unique_candidates_list = [] 
    unique_candidates_list = getlistofUniquetweets(aKeyword, 10, unique_candidates_list)
    HCPusersInfo = appendtoHCPInfo(unique_candidates_list, HCPusersInfo)
  HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)
  HCPusersInfoPD.info()  
  return HCPusersInfoPD

In [ ]:
HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)
HCPusersInfoPD.info()
HCPusersInfoPD.head()

In [ ]:
HCPusersInfo.info()

In [ ]:
def appendtoHCPInfo(listofHCPs, HCPusersInfo):
  for auser  in listofHCPs:
    test = api.lookup_users(screen_names=['DrLBoominathan'])
    for user in test:
            HCPusersInfo.append({u'screen_name':user.screen_name,
                                u'name': user.name,
                                u'description':user.description,
                                u'followers_count':user.followers_count,
                                u'statuses_count': user.statuses_count,
                                u'URL': user.url     
                                })
      
    # Generate Pandas DataFrame from list of dictionaries
  HCPusersInfoPD = pd.DataFrame.from_dict(HCPusersInfo)
    #export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 
  HCPusersInfoPD.info()
  return HCPusersInfo

# Dataset 2 :  MeSH terms with Authors specalizing in a domain

In [ ]:
articleInfo = []
articleList = []

for query in ListofMeSHqueries:

  results = pubmed.query(query, max_results=100)
  time.sleep(5)
  print(results)

  for article in results:
    # Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
    # We need to convert it to dictionary with available function
      articleDict = article.toDict()
      articleList.append(articleDict)

    # Generate list of dict records which will hold all article details that could be fetch from PUBMED API
  for article in articleList:
    #Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
      pubmedId = article['pubmed_id'].partition('\n')[0]
        # Append article info to dictionary 
      # articleInfo.append({u'pubmed_id':pubmedId,                        u'title':article['title'],                        u'keywords':article['keywords'],                       u'journal':article['journal'],})


      if (len(article['keywords'])) > 0:
        articleInfo.append({u'query':query,
                            u'MeSH Term 1': MesHTermsArray[0],
                            u'authors':article['authors'],
                            })
    
  # Generate Pandas DataFrame from list of dictionaries
articlesPD = pd.DataFrame.from_dict(articleInfo)
  #export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

#Print first 10 rows of dataframe
articlesPD.head(10)

In [ ]:
articlesPD.head(10)

In [ ]:
# Execute the query against the API
results = pubmed.query(query, max_results=5)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    print(title)
    keywords ='NONE'
    if article.keywords:
        if None in article.keywords:
            article.keywords.remove(None)
        keywords = '", "'.join(article.keywords)
    publication_date = article.publication_date
    abstract = article.abstract

    # Show information about the article
    print(
        f'{article_id} -   - \nKeywords: "{keywords}"\n\n'
    )

In [ ]:
ListofMeSHqueries

# Dataset 3:  Authors and countries , as per exisiting codebase  



In [ ]:
from google.colab import files
 

In [ ]:
query

## Upload the worldcities CSV file here  (else it may show error prompt on timeout)

In [ ]:
#upload the worldcities.csv from harddisk for now
files.upload()

### 1. PudMed dataset for the Query with [keyword1, keyword2,...] format

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul  3 15:19:12 2020

@author: ESPardilJo
"""

from pymed import PubMed
import json
import pandas as pd
import os
import re
import numpy as np

#medscape = pd.read_csv('C:/Users/ESPardilJo/OneDrive - NESTLE/Projects/WebSrapping/Medscape.csv') 
#authors = medscape.author.tolist()
rootpath = 'C:/Users/ESPardilJo/NESTLE/Mata,Karenne,BARCELONA,IT Analytics, Data and Integration - HCP/Scripts'
#rootpath = 'C:/Users/ESMataKa/OneDrive - NESTLE/Karen/HCP/Scripts'

#authors = pd.read_csv(os.path.join(rootpath, 'med-authors.csv'))
#topics = pd.read_csv(os.path.join(rootpath, 'alltopics.csv'))
#universities = pd.read_csv(os.path.join(rootpath, 'World-University-Names-Database-master/world_universities.csv'))

countries_table = pd.read_csv( 'worldcities.csv')


def look4country(string, countries_table):
    aux1 = string.lower()
    countries = list(set(countries_table.country.str.lower().tolist()))
    cities_ascii = countries_table.city_ascii.str.lower().tolist()
    iso3 = countries_table.iso3.tolist()
    iso2 = countries_table.iso2.tolist()
    if any(country in aux1 for country in countries):
        indicators = [country in aux1 for country in countries]
        country = countries[[i for i, x in enumerate(indicators) if x][0]]
        return(country)
    elif any(country in string for country in iso3):
        indicators = [country in string for country in iso3]
        country = countries_table.country[[i for i, x in enumerate(indicators) if x][0]].lower()
        return(country)
    elif any(str(country) in string for country in iso2):
        indicators = [str(country) in string for country in iso2]
        country = countries_table.country[[i for i, x in enumerate(indicators) if x][0]].lower()
        return(country)
    elif any(city in aux1 for city in cities_ascii):
        indicators = [city in aux1 for city in cities_ascii]
        country = countries_table.country[[i for i, x in enumerate(indicators) if x][0]].lower()
        return(country)


articles = pd.DataFrame()


pubmed = PubMed()

# Execute the query against the API
for query in ListofMeSHqueries:
    results = pubmed.query(query, max_results=5)
    MesHTermsArray = mydictionaryofMeSHqueries[query]

    time.sleep(5)
    
    # Loop over the retrieved articles
    for i,article in enumerate(results):
        
        article = json.loads(article.toJSON())
        text = ''
        authors2 = pd.DataFrame()
        for author2 in article['authors']:
            authors2['MeSH Query'] = query            
            authors2['MeSHTerm1'] = MesHTermsArray[0]              
            print(author2['firstname'])

            try:
                country = look4country(author2['affiliation'],countries_table)
                author2['country'] = country
            except:
                author2['country'] = ''
            try:
                author2['mail'] = re.search(r'[\w\.-]+@[\w\.-]+', author2['affiliation'], re.IGNORECASE).group(0)
                domain = re.search(r'@.+$', author2['affiliation'], re.IGNORECASE).group(0)
                author2['domain'] = re.sub('\.$', '', domain)
            except:
                author2['mail'] = ''
                author2['domain'] = ''
            try:
                where = re.search(r'[\w| ]+[,|$]', author2['affiliation'], re.IGNORECASE).group(0)
                author2['where'] = re.sub('^\d', '', re.sub(',', '', where))
            except:
                author2['where'] = ''
            try:
                uni = re.search(r'[\w| ] uni(.*) [\w| ]+[,|$]', author2['affiliation'], re.IGNORECASE).group(0)
                author2['uni'] = uni
            except:
                author2['uni'] = ''
            try:
                subject = re.search(r'of [\w| ]+$', author2['where'], re.IGNORECASE).group(0)
                author2['subject'] = re.sub('of ', '', subject)
            except:
                author2['subject'] = ''
            authors2 = pd.concat([authors2, pd.DataFrame.from_dict(author2 , orient = 'index').transpose()], axis = 0)
            text += author2['where'] + ' ' + author2['domain'] + ' '
    
        authors2['title'] = article['title']             
        authors2['abstract'] = article['abstract'] 
        if 'keywords' in article.keys():
            authors2['keywords'] = str(article['keywords'])
            keywords = article['keywords']
            if None in keywords:
                keywords.remove(None)
            text += ' '.join(keywords)
        else:
            authors2['keywords'] = ""
            
        if 'journal' in article.keys():
            authors2['journal'] = article['journal']
        elif 'publisher' in article.keys():
            authors2['journal'] = article['publisher']
            
        authors2['copyrights'] = article['copyrights']
        if 'conclusions' in article.keys():
            authors2['conclusions'] = article['conclusions'] 
        else:
            authors2['conclusions'] = ''
        if 'results' in article.keys():
            authors2['results'] = article['results']
        else:
            authors2['results'] = ''
        authors2['publication_date'] = article['publication_date']
        authors2['doi'] = article['doi']
    
        authors2['text'] = str(list(set(text.split(' '))))
        articles = pd.concat([articles,authors2], axis = 0)
        
        
countries_table.country = countries_table.country.str.lower()
countries = pd.DataFrame(countries_table[['country', 'lat', 'lng']].groupby('country').mean())
countries['country'] = countries.index

#articles = pd.merge(articles, countries, how='left', on = 'country')
        
articles.to_csv( 'pubmed_countries.csv')

In [ ]:
files.download('pubmed_countries.csv')

### 2. PudMed dataset for the Query with one [keyword] per row format .. (If there is no keyword for an article, the article is NOT considered)

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul  3 15:19:12 2020

@author: ESPardilJo
"""

from pymed import PubMed
import json
import pandas as pd
import os
import re
import numpy as np

#medscape = pd.read_csv('C:/Users/ESPardilJo/OneDrive - NESTLE/Projects/WebSrapping/Medscape.csv') 
#authors = medscape.author.tolist()
rootpath = 'C:/Users/ESPardilJo/NESTLE/Mata,Karenne,BARCELONA,IT Analytics, Data and Integration - HCP/Scripts'
#rootpath = 'C:/Users/ESMataKa/OneDrive - NESTLE/Karen/HCP/Scripts'

#authors = pd.read_csv(os.path.join(rootpath, 'med-authors.csv'))
#topics = pd.read_csv(os.path.join(rootpath, 'alltopics.csv'))
#universities = pd.read_csv(os.path.join(rootpath, 'World-University-Names-Database-master/world_universities.csv'))

countries_table = pd.read_csv( 'worldcities.csv')


def look4country(string, countries_table):
    aux1 = string.lower()
    countries = list(set(countries_table.country.str.lower().tolist()))
    cities_ascii = countries_table.city_ascii.str.lower().tolist()
    iso3 = countries_table.iso3.tolist()
    iso2 = countries_table.iso2.tolist()
    if any(country in aux1 for country in countries):
        indicators = [country in aux1 for country in countries]
        country = countries[[i for i, x in enumerate(indicators) if x][0]]
        return(country)
    elif any(country in string for country in iso3):
        indicators = [country in string for country in iso3]
        country = countries_table.country[[i for i, x in enumerate(indicators) if x][0]].lower()
        return(country)
    elif any(str(country) in string for country in iso2):
        indicators = [str(country) in string for country in iso2]
        country = countries_table.country[[i for i, x in enumerate(indicators) if x][0]].lower()
        return(country)
    elif any(city in aux1 for city in cities_ascii):
        indicators = [city in aux1 for city in cities_ascii]
        country = countries_table.country[[i for i, x in enumerate(indicators) if x][0]].lower()
        return(country)


articles = pd.DataFrame()


pubmed = PubMed()

# Execute the query against the API
for query in ListofMeSHqueries:
    results = pubmed.query(query, max_results=5)
    MesHTermsArray = mydictionaryofMeSHqueries[query]

    time.sleep(5)
    
    # Loop over the retrieved articles
    for i,article in enumerate(results):
        
      article = json.loads(article.toJSON())
      if (len(article['keywords'])) > 0:
        for keyword in article['keywords']:
          text = ''
          authors2 = pd.DataFrame()
          for author2 in article['authors']:
              authors2['MeSH Query'] = query            
              authors2['MeSHTerm1'] = MesHTermsArray[0]   
              authors2['KEYWORD'] =    keyword        
              print(author2['firstname'])

              try:
                  country = look4country(author2['affiliation'],countries_table)
                  author2['country'] = country
              except:
                  author2['country'] = ''
              try:
                  author2['mail'] = re.search(r'[\w\.-]+@[\w\.-]+', author2['affiliation'], re.IGNORECASE).group(0)
                  domain = re.search(r'@.+$', author2['affiliation'], re.IGNORECASE).group(0)
                  author2['domain'] = re.sub('\.$', '', domain)
              except:
                  author2['mail'] = ''
                  author2['domain'] = ''
              try:
                  where = re.search(r'[\w| ]+[,|$]', author2['affiliation'], re.IGNORECASE).group(0)
                  author2['where'] = re.sub('^\d', '', re.sub(',', '', where))
              except:
                  author2['where'] = ''
              try:
                  uni = re.search(r'[\w| ] uni(.*) [\w| ]+[,|$]', author2['affiliation'], re.IGNORECASE).group(0)
                  author2['uni'] = uni
              except:
                  author2['uni'] = ''
              try:
                  subject = re.search(r'of [\w| ]+$', author2['where'], re.IGNORECASE).group(0)
                  author2['subject'] = re.sub('of ', '', subject)
              except:
                  author2['subject'] = ''
              authors2 = pd.concat([authors2, pd.DataFrame.from_dict(author2 , orient = 'index').transpose()], axis = 0)
              text += author2['where'] + ' ' + author2['domain'] + ' '
      
          authors2['title'] = article['title']             
          authors2['abstract'] = article['abstract'] 
          if 'keywords' in article.keys():
              authors2['keywords'] = str(article['keywords'])
              keywords = article['keywords']
              if None in keywords:
                  keywords.remove(None)
              text += ' '.join(keywords)
          else:
              authors2['keywords'] = ""
              
          if 'journal' in article.keys():
              authors2['journal'] = article['journal']
          elif 'publisher' in article.keys():
              authors2['journal'] = article['publisher']
              
          authors2['copyrights'] = article['copyrights']
          if 'conclusions' in article.keys():
              authors2['conclusions'] = article['conclusions'] 
          else:
              authors2['conclusions'] = ''
          if 'results' in article.keys():
              authors2['results'] = article['results']
          else:
              authors2['results'] = ''
          authors2['publication_date'] = article['publication_date']
          authors2['doi'] = article['doi']
      
          authors2['text'] = str(list(set(text.split(' '))))
          articles = pd.concat([articles,authors2], axis = 0)
        
        
countries_table.country = countries_table.country.str.lower()
countries = pd.DataFrame(countries_table[['country', 'lat', 'lng']].groupby('country').mean())
countries['country'] = countries.index

#articles = pd.merge(articles, countries, how='left', on = 'country')
        
articles.to_csv( 'pubmed_countries_inAotherFormat.csv')

In [ ]:
files.download('pubmed_countries_inAotherFormat.csv')

In [ ]:
articles.tail(20)